In [ ]:
import ijson
genes_matched = []
with open('.\data\steps data output\step-3\3_matched_biological_entites', 'r', encoding='utf-8') as f:
    # Parse items one by one (adjust 'item' based on your JSON structure)
    for item in ijson.items(f, 'item'):
        genes_matched.append(item)

In [3]:
len(genes_matched)

2120

In [ ]:
import ijson
df_genes = []
with open('.\data\steps data output\step-2\3_biological_entities', 'r', encoding='utf-8') as f:
    # Parse items one by one (adjust 'item' based on your JSON structure)
    for item in ijson.items(f, 'item'):
        df_genes.append(item)

In [5]:
Gene_relations = {}
unique_key = lambda x: (x['identifier'], x['pmid'], x['type'], x['word'], x['sentence'])

for i in genes_matched:
    for x in df_genes:
        if i['Identifier'].lower() == x['identifier'].lower():
            item = {
                'identifier': x['identifier'],
                'pmid': x['PMID'],
                'type': x['type'],
                'word': x['word'],
                'sentence': x['sentence']
            }
            key = unique_key(item)
            if key not in Gene_relations:
                Gene_relations[key] = item

# Convert back to list if needed
Gene_relations = list(Gene_relations.values())

In [6]:
len(Gene_relations)

11771

In [26]:
from collections import defaultdict

# Group gene and disease relations by sentence
genes_by_sentence_1 = defaultdict(list)
genes_by_sentence_2 = defaultdict(list)

for gene1 in Gene_relations:
    genes_by_sentence_1[gene1['sentence']].append(gene1)

for gene2 in Gene_relations:
    genes_by_sentence_2[gene2['sentence']].append(gene2)

# Create unique pairs
seen_keys = set()
Gene_gene_relations = []

for sentence in genes_by_sentence_2:
    if sentence in genes_by_sentence_1:
        for gene2 in genes_by_sentence_2[sentence]:
            for gene1 in genes_by_sentence_1[sentence]:
                key1 = (gene2['pmid'], sentence, gene2['identifier'], gene1['identifier'])
                key2 = (gene2['pmid'], sentence, gene1['identifier'], gene2['identifier'])
                if key1 not in seen_keys and key2 not in seen_keys and gene2['identifier']!= gene1['identifier']:
                    seen_keys.add(key1)
                    seen_keys.add(key2)
                    Gene_gene_relations.append({
                        'pmid': gene2['pmid'],
                        'sentence': sentence,
                        'target1_identifier': gene2['identifier'],
                        'target1_type': gene2['type'],
                        'target1_word': gene2['word'],
                        'target2_identifier': gene1['identifier'],
                        'target2_type': gene1['type'],
                        'target2_word': gene1['word']
                    })


In [27]:
len(Gene_gene_relations)

8724

In [ ]:
import json
with open('.\data\steps data output\step-4\gene_gene_relations.json', "w") as json_file:
    json.dump(Gene_gene_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [ ]:
import json
with open('.\data\steps data output\step-4\relation_genes.json', "w") as json_file:
    json.dump(Gene_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [ ]:
import ijson
diseases_matched = []
with open('.\data\steps data output\step-3\3_matched_biological_entites\matched_diseases.json', 'r', encoding='utf-8') as f:
    # Parse items one by one (adjust 'item' based on your JSON structure)
    for item in ijson.items(f, 'item'):
        diseases_matched.append(item)

In [33]:
len(diseases_matched)

15472

In [ ]:
import ijson
df_diseases = []
with open('.\data\steps data output\step-2\3_biological_entities\Disease_Extraction.json', 'r', encoding='utf-8') as f:
    # Parse items one by one (adjust 'item' based on your JSON structure)
    for item in ijson.items(f, 'item'):
        df_diseases.append(item)

In [35]:
from collections import defaultdict

# Preprocess df_diseases into a lookup dictionary with uppercased identifiers
df_diseases_lookup = defaultdict(list)
for x in df_diseases:
    df_diseases_lookup[x['identifier']].append(x)

# Use a set to track seen keys and build Disease_relations list
unique_key = lambda x: (x['identifier'], x['pmid'], x['type'], x['word'], x['sentence'])
seen_keys = set()
Disease_relations = []

for i in diseases_matched:
    matches = df_diseases_lookup.get(i['Identifier'].upper(), [])
    for x in matches:
        item = {
            'identifier': x['identifier'],
            'pmid': x['PMID'],
            'type': x['type'],
            'word': x['word'],
            'sentence': x['sentence']
        }
        key = unique_key(item)
        if key not in seen_keys:
            seen_keys.add(key)
            Disease_relations.append(item)


In [36]:
len(Disease_relations)

187187

In [37]:
from collections import defaultdict

# Group gene and disease relations by sentence
diseases_by_sentence_1 = defaultdict(list)
diseases_by_sentence_2 = defaultdict(list)

for disease1 in Disease_relations:
    diseases_by_sentence_1[disease1['sentence']].append(disease1)

for disease2 in Disease_relations:
    diseases_by_sentence_2[disease2['sentence']].append(disease2)

# Create unique pairs
seen_keys = set()
Disease_disease_relations = []

for sentence in diseases_by_sentence_2:
    if sentence in diseases_by_sentence_1:
        for disease2 in diseases_by_sentence_2[sentence]:
            for disease1 in diseases_by_sentence_1[sentence]:
                key1 = (disease2['pmid'], sentence, disease2['identifier'], disease1['identifier'])
                key2 = (disease2['pmid'], sentence, disease1['identifier'], disease2['identifier'])
                if key1 not in seen_keys and key2 not in seen_keys and disease2['identifier']!= disease1['identifier']:
                    seen_keys.add(key1)
                    seen_keys.add(key2)
                    Disease_disease_relations.append({
                        'pmid': disease2['pmid'],
                        'sentence': sentence,
                        'target1_identifier': disease2['identifier'],
                        'target1_type': disease2['type'],
                        'target1_word': disease2['word'],
                        'target2_identifier': disease1['identifier'],
                        'target2_type': disease1['type'],
                        'target2_word': disease1['word']
                    })


In [38]:
len(Disease_disease_relations)

72225

In [43]:
Disease_disease_relations[2]

{'pmid': '24903423',
 'sentence': 'These tumors may arise sporadically or be associated to various syndromes, namely multiple endocrine neoplasia type 2, neurofibromatosis type 1, Von Hippel-Lindau syndrome, and hereditary paraganglioma-pheochromocytoma syndromes.',
 'target1_identifier': 'MESH:D009456',
 'target1_type': 'Disease',
 'target1_word': 'neurofibromatosis type 1',
 'target2_identifier': 'MESH:D006623',
 'target2_type': 'Disease',
 'target2_word': 'Von Hippel-Lindau syndrome'}

In [ ]:
import json
with open('.\data\steps data output\step-4\disease_disease_relations.json', "w") as json_file:
    json.dump(Disease_disease_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [ ]:
import json
with open('.\data\steps data output\step-4\relation_diseases.json', "w") as json_file:
    json.dump(Disease_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [24]:
from collections import defaultdict

# Group gene and disease relations by sentence
genes_by_sentence = defaultdict(list)
diseases_by_sentence = defaultdict(list)

for gene in Gene_relations:
    genes_by_sentence[gene['sentence']].append(gene)

for disease in Disease_relations:
    diseases_by_sentence[disease['sentence']].append(disease)

# Create unique pairs
seen_keys = set()
Disease_gene_relations = []

for sentence in diseases_by_sentence:
    if sentence in genes_by_sentence:
        for disease in diseases_by_sentence[sentence]:
            for gene in genes_by_sentence[sentence]:
                key = (disease['pmid'], sentence, disease['identifier'], gene['identifier'])
                if key not in seen_keys:
                    seen_keys.add(key)
                    Disease_gene_relations.append({
                        'pmid': disease['pmid'],
                        'sentence': sentence,
                        'target1_identifier': disease['identifier'],
                        'target1_type': disease['type'],
                        'target1_word': disease['word'],
                        'target2_identifier': gene['identifier'],
                        'target2_type': gene['type'],
                        'target2_word': gene['word']
                    })


In [25]:
len(Disease_gene_relations)

6063

In [ ]:
import json
with open('.\data\steps data output\step-4\disease_gene_relations.json', "w") as json_file:
    json.dump(Disease_gene_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [ ]:
import ijson
df_chemicals = []
with open('.\data\steps data output\step-2\3_biological_entities\Chemical_Extraction.json', 'r', encoding='utf-8') as f:
    # Parse items one by one (adjust 'item' based on your JSON structure)
    for item in ijson.items(f, 'item'):
        df_chemicals.append(item)

In [12]:
from collections import defaultdict

# Group chemical and disease relations by sentence
chemicals_by_sentence = defaultdict(list)
diseases_by_sentence = defaultdict(list)

for chemical in df_chemicals:
    chemicals_by_sentence[chemical['sentence']].append(chemical)

for disease in Disease_relations:
    diseases_by_sentence[disease['sentence']].append(disease)

# Create unique pairs
seen_keys = set()
Disease_chemical_relations = []

for sentence in diseases_by_sentence:
    if sentence in chemicals_by_sentence:
        for disease in diseases_by_sentence[sentence]:
            for chemical in chemicals_by_sentence[sentence]:
                key = (disease['pmid'], sentence, disease['identifier'], chemical['identifier'])
                if key not in seen_keys:
                    seen_keys.add(key)
                    Disease_chemical_relations.append({
                        'pmid': disease['pmid'],
                        'sentence': sentence,
                        'target1_identifier': disease['identifier'],
                        'target1_type': disease['type'],
                        'target1_word': disease['word'],
                        'target2_identifier': chemical['identifier'],
                        'target2_type': chemical['type'],
                        'target2_word': chemical['word']
                    })

In [13]:
len(Disease_chemical_relations)

13598

In [14]:
Disease_chemical_relations[0]

{'pmid': '25588595',
 'sentence': 'BACKGROUND: In nonvalvular atrial fibrillation (NVAF), rivaroxaban is used to prevent stroke and systemic embolism.',
 'target1_identifier': 'MESH:D004617',
 'target1_type': 'Disease',
 'target1_word': 'systemic embolism',
 'target2_identifier': 'MESH:D000069552',
 'target2_type': 'Chemical',
 'target2_word': 'rivaroxaban'}

In [ ]:
import json
with open('.\data\steps data output\step-4\disease_chemical_relations.json', "w") as json_file:
    json.dump(Disease_chemical_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [ ]:
import ijson
df_chromosomes = []
with open('.\data\steps data output\step-2\3_biological_entities\Chromosome_Extraction.json', 'r', encoding='utf-8') as f:
    # Parse items one by one (adjust 'item' based on your JSON structure)
    for item in ijson.items(f, 'item'):
        df_chromosomes.append(item)

In [16]:
from collections import defaultdict

# Group chromosome and disease relations by sentence
chromosomes_by_sentence = defaultdict(list)
diseases_by_sentence = defaultdict(list)

for chromosome in df_chromosomes:
    chromosomes_by_sentence[chromosome['sentence']].append(chromosome)

for disease in Disease_relations:
    diseases_by_sentence[disease['sentence']].append(disease)

# Create unique pairs
seen_keys = set()
Disease_chromosome_relations = []

for sentence in diseases_by_sentence:
    if sentence in chromosomes_by_sentence:
        for disease in diseases_by_sentence[sentence]:
            for chromosome in chromosomes_by_sentence[sentence]:
                key = (disease['pmid'], sentence, disease['identifier'], chromosome['identifier'])
                if key not in seen_keys:
                    seen_keys.add(key)
                    Disease_chromosome_relations.append({
                        'pmid': disease['pmid'],
                        'sentence': sentence,
                        'target1_identifier': disease['identifier'],
                        'target1_type': disease['type'],
                        'target1_word': disease['word'],
                        'target2_identifier': chromosome['identifier'],
                        'target2_type': chromosome['type'],
                        'target2_word': chromosome['word']
                    })

In [17]:
len(Disease_chromosome_relations)

77

In [ ]:
import json
with open('.\data\steps data output\step-4\disease_chromosome_relations.json', "w") as json_file:
    json.dump(Disease_chromosome_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [ ]:
import ijson
df_variants = []
with open('.\data\steps data output\step-2\3_biological_entities\Variant_Extraction.json', 'r', encoding='utf-8') as f:
    # Parse items one by one (adjust 'item' based on your JSON structure)
    for item in ijson.items(f, 'item'):
        df_variants.append(item)

In [29]:
from collections import defaultdict

# Group variant and disease relations by sentence
variants_by_sentence = defaultdict(list)
diseases_by_sentence = defaultdict(list)

for variant in df_variants:
    variants_by_sentence[variant['sentence']].append(variant)

for disease in Disease_relations:
    diseases_by_sentence[disease['sentence']].append(disease)

# Create unique pairs
seen_keys = set()
Disease_variant_relations = []

for sentence in diseases_by_sentence:
    if sentence in variants_by_sentence:
        for disease in diseases_by_sentence[sentence]:
            for variant in variants_by_sentence[sentence]:
                key = (disease['pmid'], sentence, disease['identifier'], variant['identifier'])
                if key not in seen_keys:
                    seen_keys.add(key)
                    Disease_variant_relations.append({
                        'pmid': disease['pmid'],
                        'sentence': sentence,
                        'target1_identifier': disease['identifier'],
                        'target1_type': disease['type'],
                        'target1_word': disease['word'],
                        'target2_identifier': variant['identifier'],
                        'target2_type': variant['type'],
                        'target2_word': variant['word']
                    })

In [46]:
len(Disease_variant_relations)

264

In [ ]:
import json
with open('.\data\steps data output\step-4\disease_variant_relations.json', "w") as json_file:
    json.dump(Disease_variant_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [18]:
from collections import defaultdict

# Group chromosome and genes relations by sentence
chromosomes_by_sentence = defaultdict(list)
genes_by_sentence = defaultdict(list)

for chromosome in df_chromosomes:
    chromosomes_by_sentence[chromosome['sentence']].append(chromosome)

for gene in Gene_relations:
    genes_by_sentence[gene['sentence']].append(gene)

# Create unique pairs
seen_keys = set()
Gene_chromosome_relations = []

for sentence in genes_by_sentence:
    if sentence in chromosomes_by_sentence:
        for gene in genes_by_sentence[sentence]:
            for chromosome in chromosomes_by_sentence[sentence]:
                key = (gene['pmid'], sentence, gene['identifier'], chromosome['identifier'])
                if key not in seen_keys:
                    seen_keys.add(key)
                    Gene_chromosome_relations.append({
                        'pmid': gene['pmid'],
                        'sentence': sentence,
                        'target1_identifier': gene['identifier'],
                        'target1_type': gene['type'],
                        'target1_word': gene['word'],
                        'target2_identifier': chromosome['identifier'],
                        'target2_type': chromosome['type'],
                        'target2_word': chromosome['word']
                    })

In [19]:
len(Gene_chromosome_relations)

194

In [21]:
Gene_chromosome_relations[0]

{'pmid': '31706190',
 'sentence': 'This study investigated whether any of the six initially discovered genomic loci associating with RLS (BTBD9, MEIS1, PTPRD, MAP2K5/SKOR1, TOX3, and an intergenic region on chromosome 2), were more strongly associated with complaints of painful versus non-painful RLS.',
 'target1_identifier': '5607',
 'target1_type': 'Gene',
 'target1_word': 'MAP2K5',
 'target2_identifier': '2',
 'target2_type': 'Chromosome',
 'target2_word': 'chromosome 2'}

In [ ]:
import json
with open('.\data\steps data output\step-4\gene_chromosome_relations.json', "w") as json_file:
    json.dump(Gene_chromosome_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [23]:
from collections import defaultdict

# Group chemical and disease relations by sentence
chemicals_by_sentence = defaultdict(list)
genes_by_sentence = defaultdict(list)

for chemical in df_chemicals:
    chemicals_by_sentence[chemical['sentence']].append(chemical)

for gene in Gene_relations:
    genes_by_sentence[gene['sentence']].append(gene)

# Create unique pairs
seen_keys = set()
Gene_chemical_relations = []

for sentence in genes_by_sentence:
    if sentence in chemicals_by_sentence:
        for gene in genes_by_sentence[sentence]:
            for chemical in chemicals_by_sentence[sentence]:
                key = (disease['pmid'], sentence, gene['identifier'], chemical['identifier'])
                if key not in seen_keys:
                    seen_keys.add(key)
                    Gene_chemical_relations.append({
                        'pmid': gene['pmid'],
                        'sentence': sentence,
                        'target1_identifier': gene['identifier'],
                        'target1_type': gene['type'],
                        'target1_word': gene['word'],
                        'target2_identifier': chemical['identifier'],
                        'target2_type': chemical['type'],
                        'target2_word': chemical['word']
                    })

In [24]:
len(Gene_chemical_relations)

1908

In [25]:
Gene_chemical_relations[0]

{'pmid': '32393603',
 'sentence': 'PURPOSE: Differentiation syndrome (DS) is a serious adverse reaction of isocitrate dehydrogenase (IDH) inhibitors ivosidenib and enasidenib in patients with (IDH)1- and IDH2-mutated acute myeloid leukemia (AML), respectively.',
 'target1_identifier': '3417',
 'target1_type': 'Gene',
 'target1_word': 'isocitrate dehydrogenase',
 'target2_identifier': 'MESH:C000627630',
 'target2_type': 'Chemical',
 'target2_word': 'ivosidenib'}

In [ ]:
import json
with open('.\data\steps data output\step-4\gene_chemical_relations.json', "w") as json_file:
    json.dump(Gene_chemical_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [30]:
from collections import defaultdict

# Group variant and disease relations by sentence
variants_by_sentence = defaultdict(list)
genes_by_sentence = defaultdict(list)

for variant in df_variants:
    variants_by_sentence[variant['sentence']].append(variant)

for gene in Gene_relations:
    genes_by_sentence[gene['sentence']].append(gene)

# Create unique pairs
seen_keys = set()
Gene_variant_relations = []

for sentence in diseases_by_sentence:
    if sentence in variants_by_sentence:
        for gene in genes_by_sentence[sentence]:
            for variant in variants_by_sentence[sentence]:
                key = (gene['pmid'], sentence, gene['identifier'], variant['identifier'])
                if key not in seen_keys:
                    seen_keys.add(key)
                    Gene_variant_relations.append({
                        'pmid': gene['pmid'],
                        'sentence': sentence,
                        'target1_identifier': gene['identifier'],
                        'target1_type': gene['type'],
                        'target1_word': gene['word'],
                        'target2_identifier': variant['identifier'],
                        'target2_type': variant['type'],
                        'target2_word': variant['word']
                    })

In [31]:
len(Gene_variant_relations)

172

In [32]:
Gene_variant_relations[0]

{'pmid': '33764904',
 'sentence': 'Compartment-specific gene expression data and immunohistochemistry were also queried.RESULTSThe identical germline mutation of the PALLD gene (NM_001166108.1:c.G154A:p.D52N) was detected in the index patient with pancreatic cancer and the tumor tissue of her sister.',
 'target1_identifier': '23022',
 'target1_type': 'Gene',
 'target1_word': 'PALLD',
 'target2_identifier': 'tmVar:c|SUB|G|154|A;HGVS:c.154G>A;VariantGroup:1;CorrespondingGene:23022;RS#:539876255;CorrespondingSpecies:9606;CA#:3135302',
 'target2_type': 'Variant',
 'target2_word': 'c.G154A'}

In [ ]:
import json
with open('.\data\steps data output\step-4\gene_variant_relations.json', "w") as json_file:
    json.dump(Gene_variant_relations, json_file, indent=4)  # indent=4 for pretty-printing

In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from neo4j import GraphDatabase
from tqdm import tqdm

# === Load REBEL model once ===
print("🔄 Loading REBEL model...")
model_name = "Babelscape/rebel-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# === Load input JSON data ===
with open("./Relation Extraction/disease_chromosome_relations.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# === Prepare inputs for batch processing ===
sentences = [f"<triplet> {item['sentence']}" for item in data]
batch_size = 4  # adjust based on your GPU/CPU capacity

# === Extract triplets from REBEL output ===
def extract_triplets(text):
    triplets = []
    if "<triplet>" not in text:
        return triplets
    for segment in text.split("<triplet>")[1:]:
        parts = segment.strip().split("<subj>")
        for part in parts:
            if "<obj>" in part:
                subject = part.split("<obj>")[0].strip()
                rest = part.split("<obj>")[1].strip()
                if "<rel>" in rest:
                    obj, rel = rest.split("<rel>")
                    triplets.append({
                        "subject": subject.strip(),
                        "relation": rel.strip(),
                        "object": obj.strip()
                    })
    return triplets

# === Run batched inference and filter ===
print("🔍 Extracting relations from sentences...")
results = []

for i in tqdm(range(0, len(sentences), batch_size)):
    batch = sentences[i:i + batch_size]
    batch_inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**batch_inputs, max_length=512, num_beams=4)
    decoded_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    for j, decoded in enumerate(decoded_batch):
        item = data[i + j]
        t1 = item["target1_word"].lower()
        t2 = item["target2_word"].lower()
        triplets = extract_triplets(decoded)

        for t in triplets:
            subj, obj = t["subject"].lower(), t["object"].lower()
            if (t1 in subj and t2 in obj) or (t2 in subj and t1 in obj):
                results.append({
                    "pmid": item["pmid"],
                    "subject": t["subject"],
                    "relation": t["relation"],
                    "object": t["object"],
                    "sentence": item["sentence"]
                })

# === Save results to JSON ===
with open("extracted_relations.json", "w", encoding="utf-8") as f_out:
    json.dump(results, f_out, ensure_ascii=False, indent=2)

print(f"\n✅ Extracted {len(results)} relations. Saved to 'extracted_relations.json'.")



